In [1]:
!git clone https://github.com/CCogS-Mx/text-preprocessing.git

Cloning into 'text-preprocessing'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 54 (delta 16), reused 44 (delta 11), pack-reused 0
Unpacking objects: 100% (54/54), 21.99 KiB | 1.37 MiB/s, done.


## Imports

In [1]:
import sys
sys.path.append('text-preprocessing/script/')
sys.path.append('Scripts/')

In [2]:
import os
import pandas as pd
import numpy as np

# Import classes
import text_preprocessing as tp
import feature_extraction as fe
import robust_classic_model as rcm
import cleaning_twitter_data as ctd

# Algoritmos de ML
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

# Vectorizadores
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data

In [3]:
main_path = 'Data/'
training = pd.read_csv(f'{main_path}homomex_training.csv').drop('index', axis=1)

In [4]:
training.head()

,tweets,label
0,"Me quise ligar a una chava ayer y no me pelo, ...",P
1,"@papaya_rockera eres un puñal, Papayita.",P
2,Magnate ofrece 130 mdd al hombre que conquiste...,P
3,Los trolebuses del desgobierno de @EPN son idi...,P
4,"En época de Hitler no se decía ""eres gay"" y, s...",P


In [5]:
prep = tp.Preprocessing('spanish')

In [6]:
training_p = prep.main_preprocess(data=training, 
                                  column='tweets', 
                                  tweet=True, 
                                  tweet_tags=True, 
                                  remove_stop_words=False, 
                                  lemmatize=False, 
                                  translate_emojis=True)

In [7]:
training_p.head()

,tweets,label
0,me quise ligar a una chava ayer y no me pelo l...,P
1,eres un punal papayita,P
2,magnate ofrece mdd al hombre que conquiste a s...,P
3,los trolebuses del desgobierno de son idiotas ...,P
4,en epoca de hitler no se decia eres gay y si e...,P


In [8]:
training_p.label = training_p.label.fillna('NR')

In [9]:
dictionary_list = [{'P': 0, 'NP': 1, 'NR': 2}]

In [10]:
training_p.label.replace(list(dictionary_list[0].keys()), 
            list(dictionary_list[0].values()), 
            inplace=True)

In [11]:
training_p.label.value_counts()

label
1    4360
2    1778
0     862
Name: count, dtype: int64

## Finding best hyperparams (TP)


In [17]:
from sklearn.model_selection import KFold, GridSearchCV

In [18]:
models = [LogisticRegression(), RandomForestClassifier(), LinearSVC(), KNeighborsClassifier()]
model_names = ['Regresión logística', 'Random Forest', 'SVM Lineal', 'KNN']
x_label_column = 'tweets'
y_label_column = 'label'
folds = 5
grid_params  = [{"C":np.logspace(-3,3,7), "penalty":["l2"], "max_iter": [1000, 10000], "random_state":[42]},
                {"n_estimators": [10, 50, 100], "max_depth":[10, 50, 100], "random_state": [42]},
                {"C":np.logspace(-3,3,7), "penalty":["l2"], "max_iter": [1000, 10000], "random_state":[42]},
                {"n_neighbors":[3], "weights":['uniform', 'distance']}]
path_to_save_models = ''
path_to_save_vectorizers = ''
vectorizers = [CountVectorizer(ngram_range = (1,1), lowercase = False), TfidfVectorizer(ngram_range = (1,1), lowercase = False)]
vectorizers_names = ['CountVectorizer', 'TfidfVectorizer']

In [20]:
for vectorizer in vectorizers:
    print(f'Embedding: {vectorizers_names[vectorizers.index(vectorizer)]}')
    X_train = vectorizer.fit_transform(training_p[x_label_column])
    y_train = training_p[y_label_column].tolist()
    for j, model in enumerate(models):
        print(f'Model {j}: {model_names[j]}')
        kFold_cv = KFold(n_splits=folds,
                        shuffle=True, 
                        random_state=42)
        param_grid = grid_params
        grid_search = GridSearchCV(model,
                                param_grid[j],
                                cv=kFold_cv.split(X_train, y_train),
                                scoring='f1_macro',
                                verbose=3,
                                return_train_score=True)
        grid_search.fit(X_train, y_train)

        print("Best parameters: ", grid_search.best_params_)
        print("Accuracy :", grid_search.best_score_)

Embedding: CountVectorizer
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.306, test=0.297) total time=   0.1s
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.316, test=0.306) total time=   0.3s
[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.302, test=0.293) total time=   0.1s
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.311, test=0.295) total time=   0.2s
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.287, test=0.294) total time=   0.2s
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.306, test=0.297) total time=   0.1s
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.316, test=0.306) total time=   0.2s
[CV 3/5] END C=0.001, max_iter=10000, penalty=l2, rando

C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/5] END C=0.1, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.960, test=0.737) total time=   0.5s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5] END C=0.1, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.957, test=0.753) total time=   0.5s


...

[CV 3/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.999, test=0.685) total time=   6.1s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 4/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.998, test=0.663) total time=   6.1s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 5/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.999, test=0.674) total time=   6.3s
Best parameters:  {'C': 1.0, 'max_iter': 1000, 'penalty': 'l2', 'random_state': 42}
Accuracy : 0.7363699056755008
Model 3: KNN
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END n_neighbors=3, weights=uniform;, score=(train=0.782, test=0.583) total time=   0.8s
[CV 2/5] END n_neighbors=3, weights=uniform;, score=(train=0.777, test=0.625) total time=   0.6s
[CV 3/5] END n_neighbors=3, weights=uniform;, score=(train=0.773, test=0.587) total time=   0.6s
[CV 4/5] END n_neighbors=3, weights=uniform;, score=(train=0.772, test=0.615) total time=   0.6s
[CV 5/5] END n_neighbors=3, weights=uniform;, score=(train=0.771, test=0.576) total time=   0.6s
[CV 1/5] END n_neighbors=3, weights=distance;, score=(train=0.999, test=0.594) total time=   0.5s
[CV 2/5] END n_neighbors=3, weights=distance;, score=(train=0.999, test=0.630) total time=   0.5s
[CV 3/5] END

In [21]:
vectorizers = [CountVectorizer(ngram_range = (1,2), lowercase = False), TfidfVectorizer(ngram_range = (1,2), lowercase = False)]
for vectorizer in vectorizers:
    print(f'Embedding: {vectorizers_names[vectorizers.index(vectorizer)]}')
    X_train = vectorizer.fit_transform(training_p[x_label_column])
    y_train = training_p[y_label_column].tolist()
    for j, model in enumerate(models):
        print(f'Model {j}: {model_names[j]}')
        kFold_cv = KFold(n_splits=folds,
                        shuffle=True, 
                        random_state=42)
        param_grid = grid_params
        grid_search = GridSearchCV(model,
                                param_grid[j],
                                cv=kFold_cv.split(X_train, y_train),
                                scoring='f1_macro',
                                verbose=3,
                                return_train_score=True)
        grid_search.fit(X_train, y_train)

        print("Best parameters: ", grid_search.best_params_)
        print("Accuracy :", grid_search.best_score_)

Embedding: CountVectorizer
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.317, test=0.308) total time=   1.2s
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.325, test=0.307) total time=   1.1s
[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.311, test=0.297) total time=   1.1s
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.323, test=0.299) total time=   1.1s
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.302, test=0.295) total time=   1.2s
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.317, test=0.308) total time=   1.2s
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.325, test=0.307) total time=   1.2s
[CV 3/5] END C=0.001, max_iter=10000, penalty=l2, rando

C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5] END C=0.1, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.996, test=0.755) total time=   0.9s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 3/5] END C=0.1, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.997, test=0.765) total time=   1.0s


...

[CV 3/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=1.000, test=0.726) total time=  15.0s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 4/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=1.000, test=0.733) total time=  14.4s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 5/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.999, test=0.729) total time=  17.5s
Best parameters:  {'C': 10.0, 'max_iter': 1000, 'penalty': 'l2', 'random_state': 42}
Accuracy : 0.7298221323032593
Model 3: KNN
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END n_neighbors=3, weights=uniform;, score=(train=0.783, test=0.593) total time=   1.8s
[CV 2/5] END n_neighbors=3, weights=uniform;, score=(train=0.781, test=0.588) total time=   1.8s
[CV 3/5] END n_neighbors=3, weights=uniform;, score=(train=0.782, test=0.568) total time=   1.8s
[CV 4/5] END n_neighbors=3, weights=uniform;, score=(train=0.773, test=0.612) total time=   1.8s
[CV 5/5] END n_neighbors=3, weights=uniform;, score=(train=0.776, test=0.587) total time=   1.9s
[CV 1/5] END n_neighbors=3, weights=distance;, score=(train=0.999, test=0.611) total time=   1.7s
[CV 2/5] END n_neighbors=3, weights=distance;, score=(train=0.999, test=0.592) total time=   1.8s
[CV 3/5] EN

In [22]:
vectorizers = [CountVectorizer(ngram_range = (1,3), lowercase = False), TfidfVectorizer(ngram_range = (1,3), lowercase = False)]
for vectorizer in vectorizers:
    print(f'Embedding: {vectorizers_names[vectorizers.index(vectorizer)]}')
    X_train = vectorizer.fit_transform(training_p[x_label_column])
    y_train = training_p[y_label_column].tolist()
    for j, model in enumerate(models):
        print(f'Model {j}: {model_names[j]}')
        kFold_cv = KFold(n_splits=folds,
                        shuffle=True, 
                        random_state=42)
        param_grid = grid_params
        grid_search = GridSearchCV(model,
                                param_grid[j],
                                cv=kFold_cv.split(X_train, y_train),
                                scoring='f1_macro',
                                verbose=3,
                                return_train_score=True)
        grid_search.fit(X_train, y_train)

        print("Best parameters: ", grid_search.best_params_)
        print("Accuracy :", grid_search.best_score_)

Embedding: CountVectorizer
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.326, test=0.300) total time=   2.7s
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.334, test=0.314) total time=   2.6s
[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.318, test=0.294) total time=   2.5s
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.331, test=0.306) total time=   2.7s
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.305, test=0.293) total time=   2.5s
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.326, test=0.300) total time=   2.8s
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.334, test=0.314) total time=   2.6s
[CV 3/5] END C=0.001, max_iter=10000, penalty=l2, rando

C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/5] END C=0.1, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.998, test=0.755) total time=   1.5s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 3/5] END C=0.1, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.999, test=0.762) total time=   1.8s


...

[CV 4/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=1.000, test=0.718) total time=  22.1s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 5/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.999, test=0.704) total time=  27.8s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best parameters:  {'C': 1000.0, 'max_iter': 1000, 'penalty': 'l2', 'random_state': 42}
Accuracy : 0.7072538338806782
Model 3: KNN
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END n_neighbors=3, weights=uniform;, score=(train=0.783, test=0.596) total time=   3.3s
[CV 2/5] END n_neighbors=3, weights=uniform;, score=(train=0.775, test=0.601) total time=   3.3s
[CV 3/5] END n_neighbors=3, weights=uniform;, score=(train=0.771, test=0.593) total time=   3.3s
[CV 4/5] END n_neighbors=3, weights=uniform;, score=(train=0.772, test=0.620) total time=   3.4s
[CV 5/5] END n_neighbors=3, weights=uniform;, score=(train=0.775, test=0.596) total time=   3.6s
[CV 1/5] END n_neighbors=3, weights=distance;, score=(train=0.999, test=0.610) total time=   3.4s
[CV 2/5] END n_neighbors=3, weights=distance;, score=(train=0.999, test=0.606) total time=   3.4s
[CV 3/5] END n_neighbors=3, weights=distance;, score=(train=1.000, test=0.607) total time=   3.5s
[CV 4/5] END n_neighbors=3, wei

## Finding best hyperparams (TP-LPTO)

In [13]:
models = [LogisticRegression(), RandomForestClassifier(), LinearSVC(), MultinomialNB(), KNeighborsClassifier()]
model_names = ['Regresión logística', 'Random Forest', 'SVM Lineal', 'NB Multinomial', 'KNN']
x_label_column = 'tweets'
y_label_column = 'label'
folds = 5
grid_params  = [{"C":np.logspace(-3,3,7), "penalty":["l2"], "max_iter": [1000, 10000], "random_state":[42]},
                {"n_estimators": [10, 50, 100], "max_depth":[10, 50, 100], "random_state": [42]},
                {"C":np.logspace(-3,3,7), "penalty":["l2"], "max_iter": [1000, 10000], "random_state":[42]},
                {},
                {"n_neighbors":[3], "weights":['uniform', 'distance']}]
path_to_save_models = ''
path_to_save_vectorizers = ''

In [ ]:
for i in range(2):
  if i == 0:
    print('Count vectorizer')
    vectorizer = CountVectorizer(ngram_range = (1,1), lowercase = False)
    oth_feats_vectorizer = CountVectorizer(ngram_range = (1,1), lowercase = False)
  else:
    print('TF-IDF')
    vectorizer = TfidfVectorizer(ngram_range = (1,1), lowercase = False)
    oth_feats_vectorizer = TfidfVectorizer(ngram_range = (1,1), lowercase = False)

  ml_homo = rcm.RobustMachineLearningModel(models=models,
                                       model_names=model_names,
                                       train_data=training_p,
                                       x_label_column=x_label_column,
                                       y_label_column=y_label_column,
                                       folds=folds,
                                       grid_params=grid_params,
                                       path_to_save_models=path_to_save_models,
                                       path_to_save_vectorizer=path_to_save_vectorizers,
                                       lemma=True,
                                       pos=True,
                                       tag=True,
                                       other_features=True,
                                       vectorizer=vectorizer,
                                       oth_feats_vectorizer=oth_feats_vectorizer,
                                       language='es')
  ml_homo.get_best_cross_validation()

n-gram range: (1, 1)
Count vectorizer


/usr/local/lib/python3.9/dist-packages/spacy/util.py:887: UserWarning: [W095] Model 'es_core_news_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.5.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  model (Model): The parent model.


Lemma: (7000, 13822)
POS: (7000, 16)
TAG: (7000, 15)
Other features: (7000, 760)
Final vector shape: (7000, 14613)
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.335, test=0.331) total time=  26.6s
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.348, test=0.336) total time=  26.2s
[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.329, test=0.307) total time=  25.0s
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.342, test=0.340) total time=  24.1s
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.327, test=0.323) total time=  24.1s
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.335, test=0.331) total time=  27.2s
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.348

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/5] END C=10.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.999, test=0.725) total time= 5.5min
[CV 4/5] END C=10.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.999, test=0.714) total time= 5.3min


...

[CV 2/5] END C=1000.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.999, test=0.711) total time=26.4min


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/5] END C=1000.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=1.000, test=0.712) total time=29.0min
[CV 4/5] END C=1000.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=1.000, test=0.704) total time=28.9min


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5] END C=1000.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.999, test=0.721) total time=31.6min
[CV 1/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.999, test=0.701) total time=35.5min
[CV 2/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.999, test=0.708) total time=40.2min
[CV 3/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=1.000, test=0.713) total time=39.1min


In [14]:
for i in range(2):
  if i == 0:
    print('Count vectorizer')
    vectorizer = CountVectorizer(ngram_range = (1,2), lowercase = False)
    oth_feats_vectorizer = CountVectorizer(ngram_range = (1,2), lowercase = False)
  else:
    print('TF-IDF')
    vectorizer = TfidfVectorizer(ngram_range = (1,2), lowercase = False)
    oth_feats_vectorizer = TfidfVectorizer(ngram_range = (1,2), lowercase = False)

  ml_homo = rcm.RobustMachineLearningModel(models=models,
                                       model_names=model_names,
                                       train_data=training_p,
                                       x_label_column=x_label_column,
                                       y_label_column=y_label_column,
                                       folds=folds,
                                       grid_params=grid_params,
                                       path_to_save_models=path_to_save_models,
                                       path_to_save_vectorizer=path_to_save_vectorizers,
                                       lemma=True,
                                       pos=True,
                                       tag=True,
                                       other_features=True,
                                       vectorizer=vectorizer,
                                       oth_feats_vectorizer=oth_feats_vectorizer,
                                       language='es')
  ml_homo.get_best_cross_validation()

Count vectorizer
Lemma: (7000, 79502)
POS: (7000, 218)
TAG: (7000, 200)
Other features: (7000, 7304)
Final vector shape: (7000, 87224)
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.387, test=0.368) total time= 1.1min
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.397, test=0.362) total time= 1.3min
[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.379, test=0.338) total time= 1.1min
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.388, test=0.377) total time= 1.2min
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.366, test=0.364) total time= 1.2min
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.387, test=0.368) total time= 1.2min
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;

C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/5] END C=1000.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.999, test=0.714) total time=12.7min


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/5] END C=1000.0, max_iter=1000, penalty=l2, random_state=42;, score=(train=0.999, test=0.730) total time=13.2min


...

[CV 3/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=1.000, test=0.682) total time=  36.5s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 4/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=1.000, test=0.687) total time=  33.6s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 5/5] END C=1000.0, max_iter=10000, penalty=l2, random_state=42;, score=(train=0.999, test=0.691) total time=  37.7s
tuned hpyerparameters :(best parameters)  {'C': 10.0, 'max_iter': 1000, 'penalty': 'l2', 'random_state': 42}
accuracy : 0.6863103279949436
Model 3: NB Multinomial
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..............., score=(train=0.285, test=0.267) total time=   3.1s
[CV 2/5] END ..............., score=(train=0.281, test=0.274) total time=   3.2s
[CV 3/5] END ..............., score=(train=0.286, test=0.268) total time=   3.2s
[CV 4/5] END ..............., score=(train=0.283, test=0.265) total time=   3.2s
[CV 5/5] END ..............., score=(train=0.283, test=0.273) total time=   3.1s
tuned hpyerparameters :(best parameters)  {}
accuracy : 0.2691606767822071
Model 4: KNN
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END n_neighbors=3, weights=uniform;, score=(train=0.690, test=0.420) total time=  11.9s
[CV 2/5]

In [15]:
for i in range(2):
  if i == 0:
    print('Count vectorizer')
    vectorizer = CountVectorizer(ngram_range = (1,3), lowercase = False)
    oth_feats_vectorizer = CountVectorizer(ngram_range = (1,3), lowercase = False)
  else:
    print('TF-IDF')
    vectorizer = TfidfVectorizer(ngram_range = (1,3), lowercase = False)
    oth_feats_vectorizer = TfidfVectorizer(ngram_range = (1,3), lowercase = False)

  ml_homo = rcm.RobustMachineLearningModel(models=models,
                                       model_names=model_names,
                                       train_data=training_p,
                                       x_label_column=x_label_column,
                                       y_label_column=y_label_column,
                                       folds=folds,
                                       grid_params=grid_params,
                                       path_to_save_models=path_to_save_models,
                                       path_to_save_vectorizer=path_to_save_vectorizers,
                                       lemma=True,
                                       pos=True,
                                       tag=True,
                                       other_features=True,
                                       vectorizer=vectorizer,
                                       oth_feats_vectorizer=oth_feats_vectorizer,
                                       language='es')
  ml_homo.get_best_cross_validation()

Count vectorizer
Lemma: (7000, 184433)
POS: (7000, 1986)
TAG: (7000, 1917)
Other features: (7000, 20581)
Final vector shape: (7000, 208917)
Model 0: Regresión logística
Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time=   7.1s
[CV 2/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time=   6.8s


C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_scorer.py", line 234, in __call__
    return self._score(
  File "C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_scorer.py", line 276, in _scor

[CV 3/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time= 3.3min
[CV 4/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time= 1.1min
[CV 5/5] END C=0.001, max_iter=1000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time= 1.2min
[CV 1/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time=   5.9s
[CV 2/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time=   4.2s
[CV 3/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time=   4.9s
[CV 4/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time=   4.5s
[CV 5/5] END C=0.001, max_iter=10000, penalty=l2, random_state=42;, score=(train=nan, test=nan) total time=   4.1s
[CV 1/5] END C=0.01, max_iter=1000, penalty=l2, random_state=42;, score=(train=nan,

C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
69 fits failed out of a total of 70.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
69 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\qk_le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\

tuned hpyerparameters :(best parameters)  {'C': 0.001, 'max_iter': 1000, 'penalty': 'l2', 'random_state': 42}
accuracy : nan
Model 1: Random Forest
Fitting 5 folds for each of 9 candidates, totalling 45 fits


MemoryError: Unable to allocate 8.72 GiB for an array with shape (5600, 208917) and data type int64